## 问财日级模板

In [ ]:

# 问财日级模板

def init(context):
    # 问句/想要选出的股票
    buy_stocks = '小市值，低估价值股'
    balck_stocks = 'st或停牌或上市20天内'     # 黑名单股票
    sell_stocks = '无'                        # 卖出股票股票
    days = 10                                 # 持仓天数
    Max_count = 1                             # 最大持股数
    is_hold_one = False
    context.T_up = 0.15
    context.T_down = - 0.3                    # 止盈止损 是否开启
    
    














    get_iwencai(buy_stocks,'want_stocks')     # 问句/想要选出的股票
    get_iwencai(balck_stocks,'black_stocks')  # 黑名单股票
    get_iwencai(sell_stocks,'sell_stocks')    # 卖出股票特征
    context.days = days                       # 持仓天数
    context.Max_count = Max_count             # 最大持股数
    context.is_hold_one = is_hold_one         # 止盈止损 是否开启
    context.T_up = 0.15
    context.T_down = - 0.3
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    context.hold_days = {}   # 计时器
    context.Cost = {}        # 买入成本
def handle_bar(context,bar_dict):
    # 止盈止损
    if context.is_hold_one == True:
        hold_on(context,bar_dict)
    # 日常买卖
    operation(context,bar_dict)
def hold_on(context,bar_dict):
    sell_stocks = []
    for stock in context.positions.keys():
        price = history(stock, ['close'], 1, '1d')['close'][-1]
        roc = price/context.Cost[stock]-1
        if roc>context.T_up or roc<context.T_down:
            sell_stocks.append(stock)
    for stock in sell_stocks:
        order_target_value(stock,0)
# 交易操作
def operation(context,bar_dict):
    sell_stocks = []
    for stock in list(context.portfolio.stock_account.positions.keys()):
        # 如果在卖出股票池中则卖出
        if stock in context.sell_stocks:
            log.info("%s在卖出股票池"%stock)
            sell_stocks.append(stock)
            continue
        if stock not in context.hold_days:
            context.hold_days[stock] = 0
        else:
            context.hold_days[stock] += 1
        if context.hold_days[stock] >= context.days:
            log.info("%s持仓到期卖出"%stock)
            sell_stocks.append(stock)
    for stock in sell_stocks:
        order_target_value(stock,0)
    
    buy_stocks = [stock for stock in context.want_stocks if stock \
    not in context.black_stocks]
    Count = max([0,context.Max_count-len(list(context.portfolio.stock_account.positions.keys()))])
    if Count>0:
        buy_stocks = buy_stocks[:Count]
        for stock in buy_stocks:
            order_percent(stock,1/context.Max_count)
            context.hold_days[stock] = 0
            h = history(stock, ['close'], 1, '1d')['close'][-1]
            context.Cost[stock] = h
            